In [251]:
import numpy as np
class memory_model:
    def __init__(self, size=4, copy=None):
        if copy is None:
            # randomize data
            self.data = np.random.randint(low=0, high=2, size=size)
        else:
            # copy data from another memory
            self.data = np.copy(copy.data)
        self.already_wrote_this_clock = False
        self.already_read_this_clock = False
        self.read_accesses = 0
    def write(self, addr, data):
        # validate that any individual memory is not written more than once per clock
        assert not self.already_wrote_this_clock
        self.data[addr] = data
        self.already_wrote_this_clock = True
    def read(self, addr):
        # validate that any individual memory is not read more than once per clock
        assert self.read_accesses == 0
        self.read_accesses += 1
        return self.data[addr]
    def observe(self, addr):
        # fake read, do not use for hardware modeling, can be used for observing/printing state in sim
        return self.data[addr]
    def clock(self): # clear assertion tracking info
        self.already_wrote_this_clock = False
        self.read_accesses = 0


Memories are ordered like:

X0Y0(A) X1Y0(B) X2Y0(C) \\
X0Y1(D) X1Y1(E) X2Y1(F) \\
X0Y2(G) X1Y2(H) X2Y2(I)

A full screen is arranged like:

ABCABC... \\ 
DEFDEF \\
GHIGHI \\
ABCABC \\
DEFDEF \\
GHIGHI

Each memory in the array can be read once per clock and written once per clock. This pixel arrangement means that for any 3x3 window on the screen, no single memory is read more than once per clock, and we can read all 9 of the pixels in the window every clock.

Two of these memories are implemented, so that one can stream results to the other, effectively a double buffer. After a full frame pass is complete, the buffers are switched so that the result is then fed back into the computation.

In [252]:
# golden patterns for control counters, hand-coded for a 6x6 screen.

# the submemory for the center pixel of the conway calculation
center_gold = np.array(
              [[0,0], [0,1], [0,2], [0,0], [0,1], [0,2],
               [1,0], [1,1], [1,2], [1,0], [1,1], [1,2], 
               [2,0], [2,1], [2,2], [2,0], [2,1], [2,2], 
               [0,0], [0,1], [0,2], [0,0], [0,1], [0,2], 
               [1,0], [1,1], [1,2], [1,0], [1,1], [1,2], 
               [2,0], [2,1], [2,2], [2,0], [2,1], [2,2]]
)

# the subaddress of the center pixel of the conway calculation, within its memory
write_gold = np.array(
             [[[0,    None, None, 1,    None, None,   None, None, None, None, None, None,   None, None, None, None, None, None,   2,    None, None, 3,    None, None,   None, None, None, None, None, None,   None, None, None, None, None, None],
               [None, 0,    None, None, 1,    None,   None, None, None, None, None, None,   None, None, None, None, None, None,   None, 2,    None, None, 3,    None,   None, None, None, None, None, None,   None, None, None, None, None, None],
               [None, None, 0,    None, None, 1,      None, None, None, None, None, None,   None, None, None, None, None, None,   None, None, 2,    None, None, 3,      None, None, None, None, None, None,   None, None, None, None, None, None]],
              [[None, None, None, None, None, None,   0,    None, None, 1,    None, None,   None, None, None, None, None, None,   None, None, None, None, None, None,   2,    None, None, 3,    None, None,   None, None, None, None, None, None],
               [None, None, None, None, None, None,   None, 0,    None, None, 1,    None,   None, None, None, None, None, None,   None, None, None, None, None, None,   None, 2,    None, None, 3,    None,   None, None, None, None, None, None],
               [None, None, None, None, None, None,   None, None, 0,    None, None, 1,      None, None, None, None, None, None,   None, None, None, None, None, None,   None, None, 2,    None, None, 3,      None, None, None, None, None, None]],
              [[None, None, None, None, None, None,   None, None, None, None, None, None,   0,    None, None, 1,    None, None,   None, None, None, None, None, None,   None, None, None, None, None, None,   2,    None, None, 3,    None, None],
               [None, None, None, None, None, None,   None, None, None, None, None, None,   None, 0,    None, None, 1,    None,   None, None, None, None, None, None,   None, None, None, None, None, None,   None, 2,    None, None, 3,    None],
               [None, None, None, None, None, None,   None, None, None, None, None, None,   None, None, 0,    None, None, 1,      None, None, None, None, None, None,   None, None, None, None, None, None,   None, None, 2,    None, None, 3   ]]]
)

# the subaddress of all pixels which are to be read for the conway calculation, within each memory
read_gold = np.array(
            [[[0,    0,    1,    1,    1,    None,   0,    0,    1,    1,    1,    None,   2,    2,    3,    3,    3,    None,   2,    2,    3,    3,    3,    None,   2,    2,    3,    3,    3,    None,   None, None, None, None, None, None],
              [0,    0,    0,    1,    1,    1,      0,    0,    0,    1,    1,    1,      2,    2,    2,    3,    3,    3,      2,    2,    2,    3,    3,    3,      2,    2,    2,    3,    3,    3,      None, None, None, None, None, None],
              [None, 0,    0,    0,    1,    1,      None, 0,    0,    0,    1,    1,      None, 2,    2,    2,    3,    3,      None, 2,    2,    2,    3,    3,      None, 2,    2,    2,    3,    3,      None, None, None, None, None, None]],
             [[0,    0,    1,    1,    1,    None,   0,    0,    1,    1,    1,    None,   0,    0,    1,    1,    1,    None,   2,    2,    3,    3,    3,    None,   2,    2,    3,    3,    3,    None,   2,    2,    3,    3,    3,    None],
              [0,    0,    0,    1,    1,    1,      0,    0,    0,    1,    1,    1,      0,    0,    0,    1,    1,    1,      2,    2,    2,    3,    3,    3,      2,    2,    2,    3,    3,    3,      2,    2,    2,    3,    3,    3   ],
              [None, 0,    0,    0,    1,    1,      None, 0,    0,    0,    1,    1,      None, 0,    0,    0,    1,    1,      None, 2,    2,    2,    3,    3,      None, 2,    2,    2,    3,    3,      None, 2,    2,    2,    3,    3   ]],
             [[None, None, None, None, None, None,   0,    0,    1,    1,    1,    None,   0,    0,    1,    1,    1,    None,   0,    0,    1,    1,    1,    None,   2,    2,    3,    3,    3,    None,   2,    2,    3,    3,    3,    None],
              [None, None, None, None, None, None,   0,    0,    0,    1,    1,    1,      0,    0,    0,    1,    1,    1,      0,    0,    0,    1,    1,    1,      2,    2,    2,    3,    3,    3,      2,    2,    2,    3,    3,    3   ],
              [None, None, None, None, None, None,   None, 0,    0,    0,    1,    1,      None, 0,    0,    0,    1,    1,      None, 0,    0,    0,    1,    1,      None, 2,    2,    2,    3,    3,      None, 2,    2,    2,    3,    3   ]]]
)

In [258]:
# basic function definitions and memory initialization
def conway (center, others):
    count = sum(others)
    if center:
        if count < 2 or count > 3:
            return 0
        else:
            return 1
    else:
        if count == 3:
            return 1
        else:
            return 0

def print_memory_contents(memories, primary):
    global clock
    print("======    ======")
    for y1 in range(2):
        for y2 in range(3):
            for x1 in range(2):
                for x2 in range(3):
                    print(memories[primary][y2][x2].observe(y1*2+x1), end='')
            print(' -> ' if y1*3+y2 == 3 else '    ', end='')
            for x1 in range(2):
                for x2 in range(3):
                    print(memories[flip(primary)][y2][x2].observe(y1*2+x1), end='')
            print()
    print("======    ======")

flip = lambda val: 1 if val == 0 else 0

memories = [[[memory_model(4) for x in range(3)] for y in range(3)]]
memories.append([[memory_model(4, copy=memories[0][y][x]) for x in range(3)] for y in range(3)])

clock = 0
trace_y, trace_x = 0, 0
primary = 0
n = 0

print_memory_contents(memories, primary)


======    ======
001001    001001
100010    100010
100011    100011
001001 -> 001001
101101    101101
011111    011111
======    ======


In [261]:
# per clock-cycle calculations

def do_clock(memories, clock):
    global primary

    local_clock = clock % 36
    center_mem = center_gold[local_clock]
    center_addr = write_gold[center_gold[local_clock][0]][center_gold[local_clock][1]][local_clock]
    center = memories[primary][center_gold[local_clock][0]][center_gold[local_clock][1]].read(center_addr)

    others = np.array([[(y, x) for x in range(3)] for y in range(3)]).reshape(9, 2)
    others = [c for c in others if np.any(c != center_gold[local_clock])]
    others = [memories[primary][c[0]][c[1]].read(read_gold[c[0]][c[1]][local_clock]) for c in others if not read_gold[c[0]][c[1]][local_clock] is None]

    result = conway(center, others)
    memories[flip(primary)][center_gold[local_clock][0]][center_gold[local_clock][1]].write(center_addr, result)

    if local_clock == 35:
        print(f"=== clock {clock}; result {result}; neighbors {sum(others)}; center {center_mem} ===")
        print_memory_contents(memories, primary)

    for y in range(3):
        for x in range(3):
            memories[primary][y][x].clock()
            memories[flip(primary)][y][x].clock()
    
    return clock + 1



n = 0
print(f'primary {primary}')
while n < 36: # do a whole frame
    clock = do_clock(memories, clock)
    n += 1
primary = flip(primary)

primary 0
=== clock 107; result 0; neighbors 2; center [2 2] ===
======    ======
000000    000000
000101    000010
010001    000101
001101 -> 001101
000011    000111
000000    000000
======    ======
